In [1]:
import pandas as pd
import docx2txt
import os
import time
import datetime

In [ ]:
word_path = "C:/Users/Velkey Artúr/Desktop/Textmining/videok/MZP/hangfileok/sövegek"
file_path = "C:/Users/Velkey Artúr/Desktop/Textmining/videok/MZP/hangfileok"

In [2]:
#leirat és audio fileok listája
mzp_word_list = [i for i in os.listdir(word_path) if i[:3] == "mzp"]
mzp_file_list = [i for i in os.listdir(file_path) if i[:3] == "mzp"]

In [3]:
#append the creation date of the audio file, this is the publication date of the youtube video
create_time_of_file = [os.path.getctime(f"{file_path}/{filename}") for filename in mzp_file_list]
#create the audio file DF
file_date_df = pd.DataFrame({"filename_m4a" : mzp_file_list, "date" : pd.to_datetime(create_time_of_file, unit = "s")})
#create new col to join on
file_date_df["filename"] = file_date_df["filename_m4a"].apply(lambda row: row[:row.find(".")])

In [6]:
#create the text DF
word_df = pd.DataFrame({"filename_word": mzp_word_list, "text": [docx2txt.process(f"{word_path}/{word}") for word in mzp_word_list]})
#new col to join on
word_df["filename"] = word_df["filename_word"].apply(lambda row: row[:row.find("e")-1])

In [8]:
word_df.head()

,filename_word,text,filename
0,mzp_1 (1)_eredeti.docx,"Pillanatnyilag semmi nem utalt rá, de időm még...",mzp_1 (1)
1,mzp_1 (10)_eredeti.docx,"Minden érdeklődőt elsősorban, de amikor nagyon...",mzp_1 (10)
2,mzp_1 (11)_eredeti.docx,Vallják. Elsősorban azonban ezt a hátunk mögöt...,mzp_1 (11)
3,mzp_1 (12)_eredeti.docx,"Országa, ahol az egész világon a második legtö...",mzp_1 (12)
4,mzp_1 (13)_eredeti.docx,Köszönöm szépen. Hát ha már egy publikus felmé...,mzp_1 (13)


In [9]:
file_date_df.head()

,filename_m4a,date,filename
0,mzp_1 (1).m4a,2021-10-09 20:25:18,mzp_1 (1)
1,mzp_1 (10).m4a,2021-09-06 17:23:19,mzp_1 (10)
2,mzp_1 (11).m4a,2021-09-07 12:31:16,mzp_1 (11)
3,mzp_1 (12).m4a,2021-09-04 10:11:39,mzp_1 (12)
4,mzp_1 (13).m4a,2021-09-20 15:02:05,mzp_1 (13)


In [90]:
#join the two dataframes
fifty_percent_text = pd.merge(word_df, file_date_df, on = "filename").loc[:,["text","filename","date"]]

In [91]:
fifty_percent_text.head()

,text,filename,date
0,"Pillanatnyilag semmi nem utalt rá, de időm még...",mzp_1 (1),2021-10-09 20:25:18
1,"Minden érdeklődőt elsősorban, de amikor nagyon...",mzp_1 (10),2021-09-06 17:23:19
2,Vallják. Elsősorban azonban ezt a hátunk mögöt...,mzp_1 (11),2021-09-07 12:31:16
3,"Országa, ahol az egész világon a második legtö...",mzp_1 (12),2021-09-04 10:11:39
4,Köszönöm szépen. Hát ha már egy publikus felmé...,mzp_1 (13),2021-09-20 15:02:05


In [92]:
path_ujsagok = "C:/Users/Velkey Artúr/Desktop/Textmining/texts/text_as_data"

In [110]:
#read in scraped newspaper datas
telex = pd.read_parquet(f"{path_ujsagok}/telex_data.parquet")
hu_24 = pd.read_parquet(f"{path_ujsagok}/24hu_data.parquet")
origo = pd.read_parquet(f"{path_ujsagok}/origo_data.parquet")
linkek_egyben = pd.read_parquet(f"{path_ujsagok}/osszes_link.parquet")

In [111]:
#join the publication date of the articles to the article datas
teljes_ujsag_text = pd.merge(linkek_egyben.loc[:,["date", "url"]], pd.concat([telex, hu_24, origo]), on = "url")

In [112]:
teljes_ujsag_text.head()

,date,url,title,text,ujsag
0,2021-11-17,https://24.hu/kultura/2021/11/17/kovacs-akos-f...,A jövő év első felében mutathatják be Kovács Á...,2022 első felében mutathatják be a mozik Kovác...,24.hu
1,2021-11-17,https://24.hu/fn/uzleti-tippek/2021/11/17/nagy...,Így készültek a vállalatok a negyedik hullámra,Bár a nagyvállalatok többsége idén őszre már n...,24.hu
2,2021-11-17,https://24.hu/kulfold/2021/11/17/oltatlanok-ko...,Oltatlanok korlátozása Ausztriában: koalíciós ...,Ausztriában hétfőn lépett életbe az a kormányi...,24.hu
3,2021-11-17,https://24.hu/tech/2021/11/17/az-arcane-vihark...,Az Arcane viharként tarolta le a filmipart (X),A Riot Games animációs sorozata még a legnagyo...,24.hu
4,2021-11-17,https://24.hu/belfold/2021/11/17/valasztasi-cs...,A kormány szerint nincs köze a választójoghoz ...,A parlament nemrég elfogadott egy módosító jav...,24.hu


In [114]:
#create basic filtering dictionary - this needs to be better made. however, this results in approximately half of the data kept
politics_dict = [
    "mzp",
    "karácsony",
    "dobrev",
    "marki-zay peter",
    "márki-zay péter",
    "marki zay peter",
    "márki zay péter",
    "dk",
    "pm",
    "párbeszéd",
    "párbeszéd magyarországért"
    "mszp",
    "jobbik",
    "pm",
    "demokratikus koalíció",
    "dobrev Klára",
    "karácsony Gergely",
    "választás",
    "előválasztás",
    "momentum",
    "magyar szocialista párt",
    "99 mozgalom",
    "miniszterelnök",
    "orbán viktor",
    "kormány",    
]

In [115]:
# I have decided to this in R
#teljes_ujsag_text["title"] = teljes_ujsag_text["title"].apply(lambda row: row.lower())
#teljes_ujsag_text["text"] = teljes_ujsag_text["text"].apply(lambda row: row.lower())

In [116]:
#creation of the filter column, if it gets TRUE, we keep it.
teljes_ujsag_text["title_contains"] = teljes_ujsag_text.title.str.contains('|'.join(politics_dict))
teljes_ujsag_text["title_contains"] = teljes_ujsag_text.title.str.contains('|'.join(politics_dict))

In [118]:
#number of articles that get TRUE on the text contains column - 
#meaning the text of the article contains at least one element of the politics_dict
teljes_ujsag_text.loc[lambda _df: _df["text_contains"] == True,:].shape

(19220, 7)

In [119]:
#telex
teljes_ujsag_text.loc[lambda _df: (_df["text_contains"] == True) & (_df["ujsag"] == "telex"),:].shape

(5520, 7)

In [120]:
#24.hu
teljes_ujsag_text.loc[lambda _df: (_df["text_contains"] == True) & (_df["ujsag"] == "24.hu"),:].shape

(7364, 7)

In [121]:
#origo
teljes_ujsag_text.loc[lambda _df: (_df["text_contains"] == True) & (_df["ujsag"] == "origo"),:].shape

(6336, 7)

In [122]:
#replacing \n in the text with nothing
fifty_percent_text["text"] = fifty_percent_text["text"].apply(lambda row: row.replace("\n"," "))
teljes_ujsag_text["text"] = teljes_ujsag_text["text"].apply(lambda row: row.replace("\n"," "))

In [124]:
#saving the data
fifty_percent_text.to_parquet(f"{path_ujsagok}/fifty_percent_mzp_speech_data.parquet")
teljes_ujsag_text.to_parquet(f"{path_ujsagok}/teljes_ujsag_text.parquet")

In [125]:
teljes_ujsag_text

,date,url,title,text,ujsag,title_contains,text_contains
0,2021-11-17,https://24.hu/kultura/2021/11/17/kovacs-akos-f...,A jövő év első felében mutathatják be Kovács Á...,2022 első felében mutathatják be a mozik Kovác...,24.hu,False,False
1,2021-11-17,https://24.hu/fn/uzleti-tippek/2021/11/17/nagy...,Így készültek a vállalatok a negyedik hullámra,Bár a nagyvállalatok többsége idén őszre már n...,24.hu,False,True
2,2021-11-17,https://24.hu/kulfold/2021/11/17/oltatlanok-ko...,Oltatlanok korlátozása Ausztriában: koalíciós ...,Ausztriában hétfőn lépett életbe az a kormányi...,24.hu,False,True
3,2021-11-17,https://24.hu/tech/2021/11/17/az-arcane-vihark...,Az Arcane viharként tarolta le a filmipart (X),A Riot Games animációs sorozata még a legnagyo...,24.hu,False,False
4,2021-11-17,https://24.hu/belfold/2021/11/17/valasztasi-cs...,A kormány szerint nincs köze a választójoghoz ...,A parlament nemrég elfogadott egy módosító jav...,24.hu,True,True
...,...,...,...,...,...,...,...
52228,2021/12/31,https://www.origo.hu/itthon/20211231-baleset-m...,Halálra gázolt egy embert a metró a Kossuth té...,Elgázolt egy embert a 2-es metró Budapest V. k...,origo,False,False
52229,2021/12/31,https://www.origo.hu/itthon/20211231-igy-valto...,Hogy lehet közlekedni majd szilveszter este?,Szilveszterkor és az újév elején nemcsak a főv...,origo,False,False
52230,2021/12/31,https://www.origo.hu/itthon/20211231-kariko-ka...,"Karikó Kataliné a 2021-es Golden Plate Award, ...","A sikerhez kap kulcsokat, aki megismeri az Aca...",origo,False,True
52231,2021/12/31,https://www.origo.hu/itthon/20211231-baleset-v...,"Vonattal ütközött egy autó Döbröközön, a sofőr...",Tehervonattal ütközött egy autó a Tolna megyei...,origo,False,False
